<a href="https://colab.research.google.com/github/nirupam15oct1/GRPO-First-Code/blob/main/Copy_of_RAG_with_Lemma_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

OPEN_API_KEY =

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving MetaGPT.pdf to MetaGPT.pdf


In [ ]:

from llama_index.core import SimpleDirectoryReader
doccuments=SimpleDirectoryReader(input_files=["MetaGPT.pdf"]).load_data()


In [ ]:
from llama_index.core.node_parser import SentenceSplitter
spilitter =SentenceSplitter(chunk_size=1024)
nodes=spilitter.get_nodes_from_documents(doccuments)

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm=OpenAI(model="gpt-3.5-turbo")
Settings.embeded_model=OpenAIEmbedding(model="text-embedding-ada-002")


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY
from llama_index.core import SummaryIndex,VectorStoreIndex # Fixed the typo: VoetorStoreIndex to VectorStoreIndex
summary_index=SummaryIndex(nodes)
vector_index=VectorStoreIndex(nodes)

In [ ]:
summary_query_engine=summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)
vector_query_engine=vector_index.as_query_engine()


In [ ]:
from llama_index.core.tools import QueryEngineTool

summary_tool=QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    )

)
vector_tool=QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from MetaGPT"
    )
)


In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

query_engine=RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[summary_tool,vector_tool],
    verbose=True
)




In [ ]:
response=query_engine.query("What is  the summary of the doccumnt?")
print(str(response))

Selecting query engine 0: The summary of the document suggests that it is useful for summarization questions related to MetaGPT..
The document details the development of MetaGPT, a meta-programming framework for multi-agent collaboration that incorporates Standardized Operating Procedures (SOPs) to enhance problem-solving capabilities. It involves role specialization, workflow management, and efficient communication protocols, along with an executable feedback mechanism to improve code generation quality during runtime. MetaGPT demonstrates state-of-the-art performance on various benchmarks, showcasing the benefits of SOPs in multi-agent collaborations. Additionally, the document discusses the development process within the MetaGPT platform, outlining user subscriptions, payment based on usage, and expanding agent capabilities. It covers the performance of MetaGPT in software development tasks, the impact of instruction levels on task outcomes, challenges such as reducing information o

In [ ]:
print(len(response.source_nodes))

34


In [ ]:
response=query_engine.query(
    " How do agents share information with other agent"
)
print(str(response))

Selecting query engine 1: Sharing information with other agents requires retrieving specific context from MetaGPT.
Agents share information with other agents by utilizing a shared message pool. This shared message pool allows all agents to exchange messages directly. Agents publish their structured messages in the pool and can also access messages from other entities transparently. This method enables any agent to retrieve required information directly from the shared pool, eliminating the need to inquire about other agents and wait for their responses, thus enhancing communication efficiency.


Tool Calling


In [ ]:
from llama_index.core.tools import FunctionTool
def add(x:int,y:int)->int:
  """ Add two integer togather"""
  return x+y

def mystery(x:int,y:int)->int:
  """Mystery function operates on top of two numbers"""
  return (x+y) + (x+y)

add_tool= FunctionTool.from_defaults(fn=add)
mystery_tool=FunctionTool.from_defaults(fn=mystery)




In [ ]:
from llama_index.llms.openai import OpenAI

llm=OpenAI(model="gpt-3.5-turbo")
response=llm.predict_and_call(
    [add_tool,mystery_tool],
    "Tell ne the oyput of the mystery function  on 2 and 9", verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
22
22


RAG with tool calling

In [ ]:
#Let understand the content of the Chunk
print(nodes[0].get_content(metadata_mode='all'))

page_label: 1
file_name: MetaGPT.pdf
file_path: MetaGPT.pdf
file_type: application/pdf
file_size: 16753634
creation_date: 2025-04-27
last_modified_date: 2025-04-27

Published as a conference paper at ICLR 2024
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jiaqi Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4, Jinlin Wang1, Zili Wang, Steven Ka Shing Yau5, Zijuan Lin4,
Liyang Zhou6, Chenyu Ran1, Lingfeng Xiao1,7, Chenglin Wu1†, J¨urgen Schmidhuber2,8
1DeepWisdom, 2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University, 4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University, 6University of Pennsylvania,
7University of California, Berkeley, 8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks

In [ ]:
from llama_index.core import VectorStoreIndex
vector_index=VectorStoreIndex(nodes)
query_engine=vector_index.as_query_engine(similarity_top_k=2)

In [ ]:
from llama_index.core.vector_stores import MetadataFilters

query_engine=vector_index.as_query_engine(
    similarity_top_k=2,
    filter=MetadataFilters.from_dicts(

    [
        {"key":"page_lebel","value":"2"}

    ]
    )
)

response=query_engine.query(
    "What are some high-level results of MetaGPT",
)

In [ ]:
print(str(response))

Some high-level results of MetaGPT include achieving an average score of 3.9, surpassing the score of ChatDev, and outperforming other general intelligent algorithms in generating executable code. Additionally, MetaGPT simplifies the process of transforming abstract requirements into detailed class and function designs, improving communication and code execution. Furthermore, MetaGPT demonstrates superior performance when collaborating with GPT-4 compared to other language models like GPT-3.5 and Deepseek Coder 33B.


In [ ]:
for n in response.source_nodes:
  print(n.metadata)

{'page_label': '23', 'file_name': 'MetaGPT.pdf', 'file_path': 'MetaGPT.pdf', 'file_type': 'application/pdf', 'file_size': 16753634, 'creation_date': '2025-04-27', 'last_modified_date': '2025-04-27'}
{'page_label': '7', 'file_name': 'MetaGPT.pdf', 'file_path': 'MetaGPT.pdf', 'file_type': 'application/pdf', 'file_size': 16753634, 'creation_date': '2025-04-27', 'last_modified_date': '2025-04-27'}


## More Compact Code

In [ ]:
from importlib.metadata import metadata
from typing import List
from llama_index.core.vector_stores import FilterCondition

def vector_query(
    query:str,
    page_numbers: List[str]
)-> str:
    """ Perform a Vector Search over an Index.

    query{str} : the string query  to be embedded .
    pagenumbers (List[str]): Filter by set of pages . LEAVE  BLANK if we  want  over all pages . Otehrwise , filter by set of specified pages .


    """
    metadata_dicts=[
        {"key": "page_label","value":p} for p in page_numbers
    ]
    query_engine=vector_index.as_query_engine(
        similarity_top_k=2,
        filter=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response=query_engine.query(query)
    return str(response)

vector_query_tool=FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query

)

In [ ]:
llm=OpenAI(model="gpt-3.5-turbo",temperature=0)
response=llm.predict_and_call(
    [vector_query_tool],
    "What are the high level rsults of MetaGPT as described on Page 2 ?",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "high level results of MetaGPT", "page_numbers": ["2"]}
=== Function Output ===
MetaGPT achieves high-level results by outperforming all preceding approaches in both HumanEval and MBPP benchmarks. It demonstrates superior performance when collaborating with GPT-4, significantly improving the Pass @k in the HumanEval benchmark compared to GPT-4. The pass rates for MetaGPT on the MBPP and HumanEval benchmarks are 85.9% and 87.7%, respectively.


In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index=SummaryIndex(nodes)
summary_query_engine=summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True
)
summary_tool=QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    )
)


In [ ]:
response=llm.predict_and_call(
    [vector_query_tool,summary_tool],
    "What are the MetaGPT comparisons with ChatDev distributed page 8 ?",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "MetaGPT comparisons with ChatDev", "page_numbers": ["8"]}
=== Function Output ===
MetaGPT outperforms ChatDev on the SoftwareDev dataset in various aspects such as executability, running times, token usage, code statistics, productivity, and human revision cost. MetaGPT achieves a higher score in executability, takes less time to generate code, uses more tokens but is more efficient in generating code lines, and has lower human revision costs compared to ChatDev. Additionally, MetaGPT demonstrates autonomous software generation capabilities and highlights the benefits of using SOPs in collaborations between multiple agents.


# Building Agentic frame work which ans wer complex Qiestion

In [ ]:
from llama_index.core.agent  import FunctionCallingAgentWorker
from llama_index.core.agent import  AgentRunner

agent_worker=FunctionCallingAgentWorker.from_tools(
    [vector_tool,summary_tool],
    llm=llm,
    verbose=True
)
agent=AgentRunner(agent_worker)


In [ ]:
response=agent.query(
    "Tell   me about the agent role in MetaGPT,"
    'and then how they communicate with each other '
)

Added user message to memory: Tell   me about the agent role in MetaGPT,and then how they communicate with each other 
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "What is the agent role in MetaGPT?"}
=== Function Output ===
The agent role in MetaGPT encompasses a range of responsibilities within the software development process, including generating product requirement documents, designing system architecture, breaking down projects into tasks, executing designated classes and functions, formulating test cases, and ensuring high-quality code through code reviews and bug identification. Each agent within MetaGPT performs specific functions that contribute to the overall software development process.
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "How do agents communicate with each other in MetaGPT?"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces,

Building agentic framework with much more user control

In [ ]:
task=agent.create_task(
    "Tell me about the agent role in MetaGPT,"
    'and then how they communicate with each other '
)


In [ ]:
step_outputs=agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent role in MetaGPT,and then how they communicate with each other 
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "What is the agent role in MetaGPT?"}
=== Function Output ===
The agent roles in MetaGPT encompass a range of responsibilities within the software development process, including generating product requirement documents, designing system architecture, breaking down projects into tasks, executing designated classes and functions, formulating test cases, and ensuring stringent code quality through collaborative efforts.
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "How do agents communicate with each other in MetaGPT?"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces, a publish-subscribe mechanism, and a shared message pool. This allows them to exchange structured messages directly, publish m

In [ ]:
completed_steps=agent.get_completed_steps(task.task_id)
print(f"Num completed for task{task.task_id} {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task3cebd300-e8a1-4225-9481-5726210e1648 1
The agent roles in MetaGPT encompass a range of responsibilities within the software development process, including generating product requirement documents, designing system architecture, breaking down projects into tasks, executing designated classes and functions, formulating test cases, and ensuring stringent code quality through collaborative efforts.


In [ ]:
upcoming_stpes=agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id} {len(upcoming_stpes)}")
upcoming_stpes[0]

Num upcoming steps for task 3cebd300-e8a1-4225-9481-5726210e1648 1


TaskStep(task_id='3cebd300-e8a1-4225-9481-5726210e1648', step_id='3e2af430-9cf6-4ff1-aeb0-013f33f8cf04', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)